# 3. Wrapping a Smolagents Agent into an Travel Guidance ACP Server

We will now create another ACP agent built with Smolagents. This second agent will be able to search the web to handle travel related questions for customers.  We will wrap the agent definition in an ACP server similarly to how we did with the first RAG agent.

## 3.1. Wrap the Travel Research Agent in ACP Server

To define our agent, we will use [CodeAgent](https://smolagents.org/docs/agents-guided-tour/) from the Smolagents library. This type of agent writes and executes Python code at every step. For this agent, we will use two tools provided by Smolagents:

- `DuckDuckGoSearchTool`: performs a web search using DuckDuckGo browser
- `VisitWebpageTool`: can visit and extract content from web pages

The agent is wrapped in an ACP server using `@server.agent()` decorator. The server will also run locally using a different port number: 8001. Run the following cell to save the agent as `smolagents_server.py` under `travel_acp_project`.

In [1]:
%%writefile ../travel_acp_project/src/smolagents_server.py
from collections.abc import AsyncGenerator
from acp_sdk.models import Message, MessagePart
from acp_sdk.server import Context, RunYield, RunYieldResume, Server
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, VisitWebpageTool
import logging
from dotenv import load_dotenv

load_dotenv()

server = Server()

# Configure the LLM
model = LiteLLMModel(
    model_id="openai/gpt-4",
    max_tokens=2048
)

@server.agent()
async def travel_research_agent(input: list[Message], context: Context) -> AsyncGenerator[RunYield, RunYieldResume]:
    """
    This is a CodeAgent that supports travelers with up-to-date information about travel destinations,
    visa requirements, best seasons to visit, and real-time tourism insights. It uses web search and
    web page browsing tools to answer travel-related questions.
    """
    agent = CodeAgent(
        tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
        model=model
    )

    prompt = input[0].parts[0].content
    response = agent.run(prompt)

    yield Message(parts=[MessagePart(content=str(response))])

if __name__ == "__main__":
    server.run(port=8000)

Overwriting ../travel_acp_project/src/smolagents_server.py


## 3.2. Adding MCP to the Travel Guidance Server

We will add another agent to the Travel Guidance Server that uses MCP to get access to tools:
- We will first define an MCP server using `FastMCP` that runs using the `stdio` transport (i.e., it runs locally) and exposes one tool;
- We will then update the travel guidance server to include a second agent;
- We will define the agent using the ToolCallingAgent of Smolagents and pass to it the tool exposed by the MCP server;
- We will finally activate the agent and interact with it using an ACP client.

Run this cell to create the file `mcpserver.py` inside the folder `travel_acp_project`. The dependencies in the `travel_acp_project` have been also updated so we can run this MCP server using `uv run`.

In [2]:
%%writefile ../travel_acp_project/src/mcpserver.py
from colorama import Fore
from mcp.server.fastmcp import FastMCP
import json
import requests

mcp = FastMCP("travelagencyserver")

@mcp.tool()
def list_travel_agencies(city: str) -> str:
    """
    This tool returns a list of travel agencies located in the given city.

    Args:
        city (str): The city name to search agencies for. Example: "Hyderabad"

    Returns:
        str: A list of travel agencies matching the city.
        Example: '[{"name": "GlobeTrotters Hyderabad", "rating": 4.6, "contact": "9876543210"}, ...]'
    """
    url = 'https://raw.githubusercontent.com/vinayak-shanawad/Travel-Planning-ACP/main/travel_agencies.json'
    resp = requests.get(url)
    agencies = json.loads(resp.text)

    matched = [a for a in agencies if a['city'].lower() == city.lower()]
    return json.dumps(matched, indent=2)

if __name__ == "__main__":
    mcp.run(transport="stdio")


Overwriting ../travel_acp_project/src/mcpserver.py


## 3.3. Updating the Travel Guidance Server to use MCP

We will add the definition of the second agent `travel_agency_agent` which helps users find travel agencies near them. Note that for the MCP server, the command is defined as `uv run`.

In [3]:
%%writefile ../travel_acp_project/src/smolagents_server.py
from collections.abc import AsyncGenerator
from acp_sdk.models import Message, MessagePart
from acp_sdk.server import Context, RunYield, RunYieldResume, Server
from smolagents import (
    CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, VisitWebpageTool,
    ToolCallingAgent, ToolCollection
)
from mcp import StdioServerParameters

server = Server()

model = LiteLLMModel(
    model_id="openai/gpt-4",
    max_tokens=2048
)

server_parameters = StdioServerParameters(
    command="uv",
    args=["run", "src/mcpserver.py"],
    env=None,
)

@server.agent()
async def travel_research_agent(input: list[Message], context: Context) -> AsyncGenerator[RunYield, RunYieldResume]:
    """
    This is a CodeAgent that supports travelers with up-to-date information about travel destinations,
    visa requirements, best seasons to visit, and real-time tourism insights. It uses web search and
    web page browsing tools to answer travel-related questions.
    """
    agent = CodeAgent(
        tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
        model=model
    )

    prompt = input[0].parts[0].content
    response = agent.run(prompt)

    yield Message(parts=[MessagePart(content=str(response))])

@server.agent()
async def travel_agency_agent(input: list[Message]) -> AsyncGenerator[RunYield, RunYieldResume]:
    "A Travel Agency Agent that helps users find travel agencies based on their city using an MCP tool."
    with ToolCollection.from_mcp(server_parameters, trust_remote_code=True) as tool_collection:
        agent = ToolCallingAgent(tools=tool_collection.tools, model=model)
        prompt = input[0].parts[0].content
        response = agent.run(prompt)
        yield Message(parts=[MessagePart(content=str(response))])

if __name__ == "__main__":
    server.run(port=8000)


Overwriting ../travel_acp_project/src/smolagents_server.py


## 3.4. Run the Travel Guidance ACP Server

Now to activate our configured ACP agent, we also need to run our agent from another terminal.

Open the second terminal by running the following commands:
- `cd travel_acp_project`: navigate to directory where we created `travel_acp_project`.
- `uv venv`: to create a virtual environment
- `uv sync`: to install the dependencies defined in `pyproject.toml` file.
- `.venv\Scripts\activate`: to activate the virtual environment.
- Type `uv run python src/smolagents_server.py` to run the server and activate our ACP agent.

## 3.5. Sequentially Chaining the Agent Calls

Now that we have two activated ACP servers, we will create a sequential workflow by sequentially chaining the three agents as follows:

1. 🧭 Travel Guidance ACP Server
    - Hosts:
        - Travel Agency Agent: Helps users find travel agencies based on their city using an MCP-based tool.
        - Travel Research Agent: Handles general travel queries like visa requirements, best time to visit, weather, etc.

2. 🧳 Travel Package ACP Server
    - Hosts: Travel Package Agent
    - Purpose: Answers user queries about travel itineraries, prices, and inclusions using travel brochures (PDFs or CSVs).

<img src="./images/Sequential_Workflow.jpg">

🔁 Workflow Steps

1. Step 1: Travel Agency Agent (on Travel Guidance ACP Server)

    Query:
    "I am based in Hyderabad. Are there any travel agencies near me?"

    Response:
    "Yes, Thrillophilia, GT Holidays, RL Tours & Travels operate in Hyderabad."

2. Step 2: Travel Research Agent (on Travel Guidance ACP Server)

    Query (with context):
    "Based on agencies like Thrillophilia, what is the best time to go for a Europe trip from Hyderabad?"

    Response:
    "Spring (April–June) and Autumn (September–October) are ideal times to visit Europe from Hyderabad based on travel trends, weather, and crowd levels."

3. Step 3: Travel Package Agent (on Travel Package ACP Server)

    Query (with full context):
    "Given that April–June is ideal and agencies like Thrillophilia operate from Hyderabad, which Europe travel packages from Hyderabad include Switzerland?"
    
    Response:
    "The 'Best of Switzerland & Italy' 9-day package by Thrillophilia is available from Hyderabad and includes visits to Lucerne, Zurich, and the Alps."

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from acp_sdk.client import Client
import asyncio
from colorama import Fore

async def run_travel_workflow() -> None:
    # Client 1: Travel Guidance ACP Server (Agent 1 & 2)
    # Client 2: Travel Package ACP Server (Agent 3)
    async with Client(base_url="http://localhost:8000") as guidance, Client(base_url="http://localhost:8001") as package:

        # Step 1: Ask travel agency agent for agencies in Hyderabad
        run1 = await guidance.run_sync(
            agent="travel_agency_agent",
            input="I am based in Hyderabad. Are there any travel agencies near me?"
        )
        agency_response = run1.output[0].parts[0].content
        print(Fore.CYAN + "TRAVEL AGENCY AGENT RESPONSE:\n" + agency_response + Fore.RESET)

        # Step 2: Ask travel research agent using agency names as context
        run2 = await guidance.run_sync(
            agent="travel_research_agent",
            input=f"Based on these agencies: {agency_response}\nWhat is the best time to go for a Europe trip from Hyderabad?"
        )
        research_response = run2.output[0].parts[0].content
        print(Fore.LIGHTMAGENTA_EX + "TRAVEL RESEARCH AGENT RESPONSE:\n" + research_response + Fore.RESET)

        # Step 3: Ask travel package agent using previous responses as context
        combined_context = f"{agency_response}\n{research_response}"
        run3 = await package.run_sync(
            agent="travel_package_agent",
            input=f"Context: {combined_context}\nWhich Europe travel packages from Hyderabad include Switzerland?"
        )
        package_response = run3.output[0].parts[0].content
        print(Fore.YELLOW + "TRAVEL PACKAGE AGENT RESPONSE:\n" + package_response + Fore.RESET)

In [6]:
# Run the workflow
asyncio.run(run_travel_workflow())

TRAVEL AGENCY AGENT RESPONSE:
Yes, there are several travel agencies available in Hyderabad. Here are a few:
1. [GT Holidays Hyderabad](https://www.gtholidays.in/hyderabad/), Rating: 4.7
2. [RL Tours & Travels](https://rltours.in/), Rating: 4.5
3. [IMAD Travel](https://imadtravel.com/), Rating: Not Available
4. StarEdge Holidays, Rating: 4.6, Contact: Not Available
5. [Thrillophilia](https://www.thrillophilia.com), Rating: Not Available
TRAVEL RESEARCH AGENT RESPONSE:
The best time to go for a Europe trip from Hyderabad is during the spring and fall shoulder seasons - from mid-April to mid-June and from September to October.
TRAVEL PACKAGE AGENT RESPONSE:
Here are the Europe travel packages from Hyderabad that include Switzerland:

1. "[Best of Switzerland & Italy | Roman Streets to Snowy Alps](https://www.thrillophilia.com/tours/best-of-switzerland-italy-tour-from-india)": Duration of 9 days & 8 nights for INR 1,43,200.
  
2. "[Scandinavian Snow Escape | Northern Nights & Winter Light